# Compute reference probabilities with Monte-Carlo

In this example, we perform a reliability benchmark based on a large Monte-Carlo sample. In order to limit the elapsed time, we consider a limited elapsed time for each problem. In order to get the best possible accuracy within this time limit, we set the coefficient of variation to zero.

In [1]:
import openturns as ot
import otbenchmark as otb
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [2]:
problemslist = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(problemslist)
numberOfProblems

26

In [3]:
coefficientOfVariation = 0.0
maximumOuterSampling = 10 ** 6
blockSize = 10 ** 0  # 10 ** 4 for real simulations
blockSize

10000

In [4]:
confidenceLevel = 0.95
maximumDurationSeconds = 5 * 60.0

In [5]:
totalDurationMinutes = numberOfProblems * maximumDurationSeconds / 60.0
totalDurationMinutes

130.0

In [6]:
model_names = [problemslist[i].getName() for i in range(numberOfProblems)]
metrics = ["PF", "N. function calls", "PMin", "PMax", "C.O.V.", "Digits", "Time (s)"]
resultArray = np.zeros((numberOfProblems, len(metrics)))
for i in tqdm(range(numberOfProblems)):
    startTime = time.time()
    problem = problemslist[i]
    name = problem.getName()
    event = problem.getEvent()
    g = event.getFunction()
    factory = otb.ProbabilitySimulationAlgorithmFactory()
    algo = factory.buildMonteCarlo(problem)
    algo.setMaximumOuterSampling(maximumOuterSampling)
    algo.setBlockSize(blockSize)
    algo.setMaximumCoefficientOfVariation(coefficientOfVariation)
    timer = ot.TimerCallback(maximumDurationSeconds)
    algo.setStopCallback(timer)
    initialNumberOfCall = g.getEvaluationCallsNumber()
    algo.run()
    result = algo.getResult()
    numberOfFunctionEvaluations = g.getEvaluationCallsNumber() - initialNumberOfCall
    computedProbability = result.getProbabilityEstimate()
    confidenceLength = result.getConfidenceLength(confidenceLevel)
    pmin = computedProbability - 0.5 * confidenceLength
    pmax = computedProbability + 0.5 * confidenceLength
    cov = result.getCoefficientOfVariation()
    if cov > 0.0:
        expectedDigits = -np.log10(cov) - 1.0
    else:
        expectedDigits = 0.0
    stopTime = time.time()
    elapsedTime = stopTime - startTime
    resultArray[i][0] = computedProbability
    resultArray[i][1] = numberOfFunctionEvaluations
    resultArray[i][2] = pmin
    resultArray[i][3] = pmax
    resultArray[i][4] = cov
    resultArray[i][5] = expectedDigits
    resultArray[i][6] = elapsedTime

100%|██████████| 26/26 [2:10:00<00:00, 300.01s/it]  


In [7]:
df = pd.DataFrame(resultArray, index=model_names, columns=metrics)
df

,PF,N. function calls,PMin,PMax,C.O.V.,Digits,Time (s)
RP8,7.908179e-04,2.413400e+08,7.872713e-04,7.943646e-04,0.002288,1.640503,300.011341
RP14,7.708905e-04,7.441900e+08,7.688964e-04,7.728846e-04,0.001320,1.879484,300.004585
RP22,4.207357e-03,1.495610e+09,4.204076e-03,4.210637e-03,0.000398,2.400308,300.000947
RP24,2.860848e-03,1.644700e+09,2.858266e-03,2.863429e-03,0.000460,2.336891,300.001723
RP25,4.175883e-05,1.567860e+09,4.143895e-05,4.207871e-05,0.003908,1.408015,300.015279
RP28,1.315725e-07,1.839290e+09,1.149947e-07,1.481503e-07,0.064286,0.191886,300.000353
RP31,3.227556e-03,1.787260e+09,3.224926e-03,3.230186e-03,0.000416,2.381211,300.000409
RP33,2.574817e-03,1.430700e+09,2.572190e-03,2.577443e-03,0.000520,2.283686,300.000557
RP35,3.478964e-03,1.415140e+09,3.475896e-03,3.482032e-03,0.000450,2.346860,300.002611
RP38,8.059349e-03,7.765700e+08,8.053061e-03,8.065638e-03,0.000398,2.399976,300.003572


In [8]:
df.to_csv("reliability_compute_reference_proba.csv")